In [1]:
# Utils
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt

import numpy as np
import time
#import _pickle as pickle
import pickle
import re, sys, unidecode
#import unidecode


# Representation
import nltk
from nltk.corpus import stopwords
import scikitplot.plotters as skplt

import wordbatch
from wordbatch.extractors import WordBag, WordHash
from wordbatch.models import FTRL

from tensorflow.contrib.learn import DNNClassifier
from tensorflow.contrib.learn import DNNEstimator
#from tecnosmartlib import DataObject

/root/anaconda3/lib/python3.6/site-packages/tensorflow/python/_pywrap_tensorflow_internal.so | _pywrap_tensorflow_internal
/root/anaconda3/lib/python3.6/site-packages/tensorflow/contrib/lite/toco/python/_tensorflow_wrap_toco.so | _tensorflow_wrap_toco


In [2]:
#plt.style.use('fivethirtyeight')

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Load Data

In [3]:
dfNoticias = pd.read_pickle("dfNoticias")

# pre procesing

Cleaning data, select clases

In [4]:
print('Selecting useful information...', end='')
_map = {}
_map["Cuerpo"] = []
_map["Seccion"] = []
_map["Seccion2"] = []
_map["Seccion3"] = []

count = 0
for index, row in dfNoticias.iterrows():
    if (row["Seccion_1"] != None and row["Cuerpo"] != None):
        _map["Cuerpo"].append(row["Cuerpo"]) 
        _map["Seccion"].append(row["Seccion_1"])
        _map["Seccion2"].append(row["Seccion_2"])
        _map["Seccion3"].append(row["Seccion_3"])
        if row["Seccion_1"] == row["Seccion_2"] and row["Seccion_2"] == row["Seccion_3"]:
            count += 1
        if row["Seccion_1"] == row["Seccion_2"] and row["Seccion_3"] == None:
            count += 1
        if row["Seccion_1"] == row["Seccion_3"] and row["Seccion_2"] == None:  
            count += 1 
        if row["Seccion_2"] == None and row["Seccion_3"] == None:
            count += 1      
print('done.')



Selecting useful information...done.


In [5]:
# categories = list(set(_map["Seccion"]))

# def mapping(inputEle):
#     for element in categories:
#     for idx, element in enumerate(categories):
#         if element == in     



# map(function_to_apply, list_of_inputs)

In [6]:
# print( len(_map['Seccion2']) )
# print(count)

In [7]:
df = pd.DataFrame(_map)
numberOfDuplicates = df.shape[0]
print('Number of data before dropping duplicates: {}'.format(df.shape[0]))
df = df.drop_duplicates(inplace= False)
df.reset_index(drop=True, inplace= True)
numberOfDuplicates -= df.shape[0]
print('Number of data after dropping duplicates : {}'.format(df.shape[0]))
print('\nNumber of duplicated data : {}'.format(numberOfDuplicates))

#df.head()

Number of data before dropping duplicates: 250339
Number of data after dropping duplicates : 238931

Number of duplicated data : 11408


In [8]:
X_untransformed = df['Cuerpo'].reset_index(drop=True)
y_untransformed = df['Seccion'].reset_index(drop=True)

assert X_untransformed.shape[0] == y_untransformed.shape[0], 'X and y dimenssion must be the same.'
print('Number of data: {}'.format(X_untransformed.shape[0]))

Number of data: 238931


In [9]:
classes = df.groupby('Seccion')['Cuerpo'].nunique()

print(classes)
nClasses = classes.shape[0]

#print('\nNumber of classes: {}'.format(nClasses))

Seccion
Corporativo        82
Cultura          3128
Deportes        76458
Economía         6750
Entretención    29438
Mundo           31664
País            75180
Sociedad         9079
Tecnología       7130
Name: Cuerpo, dtype: int64


# Feature Extraccion

### Data representation

TFID calculation

In [10]:

start = time.time()
print("start")


spanish_stopwords = stopwords.words('spanish')
def normalize_text(text):
    return u" ".join([x for x in [y for y in text.lower().strip().split(" ")] 
                      if len(x) > 1 and x not in spanish_stopwords])

X_untransformed = X_untransformed
n_docs = X_untransformed.shape[0]
n_cpu = 20

batch_size = int(n_docs/n_cpu)

wb = wordbatch.WordBatch(normalize_text, 
                         extractor=(WordBag, {"hash_ngrams": 1, "hash_ngrams_weights": [1.0, 1.0],
                                              "hash_size": 2**28, "norm": "l2", "tf": 1.0,
                                              "idf": 1.0}), procs=n_cpu, n_words=1000, minibatch_size=batch_size)
wb.dictionary_freeze = True
word_comment = wb.fit_transform(list(X_untransformed),reset= False)
X_transformed = word_comment[:, np.array(np.clip(word_comment.getnnz(axis=0) - 1, 0, 1), dtype = bool)]

end = time.time()
print("TFIDF end time :" + str(end - start) )


X = X_transformed
print('Number of features: {}'.format(X.shape[1]))

start
Normalize text
Extract wordbags
TFIDF end time :100.51729583740234
Number of features: 1000


### Output

In [11]:
class DataObject():
    def __init__(self, x, y):
        self.X = x
        self.Y = y

        try:
            self.Y.reset_index(drop=True, inplace=True)
        except:
            pass

        self.X_train = self.X
        self.Y_train = self.Y

        self.X_test  = None
        self.Y_test  = None

        self.X_test_dense = None

        self.k = 0
    def next_train_batch(self, batch_size):
        indexes = np.random.choice(self.X_train.shape[0], size=batch_size, replace=False)
        return self.X_train[indexes], self.Y_train.reindex(indexes)

    def set_train_test(self,train_fraction = None, test_fraction = None, dense=True):
        # Tests for create subsets
        if train_fraction == None and test_fraction == None:
            train_fraction = 0.8
        elif train_fraction == None:
            assert 0 <= test_fraction and test_fraction < 1, 'Test fraction must be in [0,1)'
            train_fraction = 1 - test_fraction
        else:
            assert 0 < train_fraction and train_fraction <= 1, 'Train fraction must be in (0,1]'

        train_indices = np.random.choice(self.X.shape[0], round(train_fraction*self.X.shape[0]), replace=False)
        test_indices = np.array(list(set(range(self.X.shape[0])) - set(train_indices)))

        self.X_train = self.X[train_indices]
        self.Y_train = self.Y.reindex(train_indices)
        self.Y_train = self.Y_train.reset_index()

        self.X_test = self.X[test_indices]
        self.Y_test = self.Y.reindex(test_indices)
        self.Y_test = self.Y_test.reset_index()
        if dense:
            self.X_test_dense = self.X_test.todense()
        return True


Y = y_untransformed
Y = Y.reset_index(drop=True)

def remove_accents(a):
    return unidecode.unidecode(a)

data = DataObject(X,Y.apply(remove_accents))

labels = [remove_accents(x) for x in list(data.Y.unique())]
#print(labels)

data.set_train_test(0.8)
#print(data.Y_train.reset_index(drop=True))

True

# Model

### DNN graph generation

In [128]:
tf.set_random_seed(42)

feature_columns = [tf.contrib.layers.real_valued_column('x', dimension=1000)]
#feature_columns = [tf.contrib.layers.real_valued_column("", dimension=4)]


classifier = DNNEstimator(                                
                           head = tf.contrib.learn.multi_label_head(n_classes=9,weight_column_name='class_weights' ) , feature_columns=feature_columns,
                           hidden_units=[2000, 1000, 100],
                           dropout=0.1,
                           #model_dir = './model-parallel/seccion_dnn',
                           model_dir = './tmp2',
                           config = tf.contrib.learn.RunConfig(save_checkpoints_steps = 500,
                           save_checkpoints_secs = None)
                          )


#tensorboard --logdir=./tmp2
# 10.20.0.114:6006

INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f799dc3e940>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': None, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': 500, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': './tmp2'}


### functions graph tensorflow

In [129]:
epochs = 100000

# Define the test inputs
def get_train_inputs():
    print("Paso por aca.........................")
    X_ = data.X_train
    Y_ = data.Y_train
    X_ = X_.todense()
    x = X_
    Y_ = Y_['Seccion'].reset_index(drop=True)
    #print(Y_)
    from sklearn.preprocessing import LabelBinarizer    
    #LabelBinarizer().fit(range(9)).transform(print(Y_))
#     y = y.apply(remove_accents)
#     y = y.values

    y = Y_.values
    #print(type(y))
    
    # computing lengths of each class
    sizes = {label: Y_[Y_ == label].shape[0] for label in labels}

    # creating weights for each sample
    scale_factor = 10e3
    weights = np.asarray([1.0/sizes[label] for label in y])
    weights = scale_factor*weights
    weights = weights[:,np.newaxis]
    print(weights)
    
    #print('weights shape: {}'.format(weights.shape))
    
    
    labels2 = list( set(y) )
    def to_index(strInput):
        tmp0 = [0]*len(labels2) 
        tmp0[labels2.index(strInput)] = 1
        return tmp0

    y = list( map(lambda x: to_index(x),y) )    

    #print(np.array(y))
    #print(y[:,np.newaxis])
    
    dataset = tf.estimator.inputs.numpy_input_fn({'x': x, 'class_weights': weights}, np.array(y), shuffle=True, batch_size=500, num_epochs=epochs)
    
    #dataset = tf.estimator.inputs.numpy_input_fn({'x': x, 'class_weights': weights}, y[:,np.newaxis], shuffle=True, batch_size=500, num_epochs=epochs)
    return dataset

def get_test_inputs():
    X_ = data.X_test_dense
    Y_ = data.Y_test
#     X_ = X_.todense()
    x = X_
    Y_ = Y_['Seccion'].reset_index(drop=True)
    y = Y_.values
    
    # computing lengths of each class
    sizes = {label: Y_[Y_ == label].shape[0] for label in labels}
#     print(sizes)

    # creating weights for each sample
    scale_factor = 10e3
    weights = np.asarray([1.0/sizes[label] for label in y])
    weights = scale_factor*weights
    weights = weights[:,np.newaxis]
    print('weights shape: {}'.format(weights.shape))

    labels2 = list( set(y) )
    def to_index(strInput):
        tmp0 = [0]*len(labels2) 
        tmp0[labels2.index(strInput)] = 1
        return tmp0

    y = list( map(lambda x: to_index(x),y) )    
    dataset = tf.estimator.inputs.numpy_input_fn({'x': x, 'class_weights': weights}, np.array(y), shuffle=False)    
    
    #dataset = tf.estimator.inputs.numpy_input_fn({'x': x, 'class_weights': weights}, y[:,np.newaxis], shuffle=False)
    return dataset


In [14]:
# tf.set_random_seed(42)

# feature_columns = [tf.contrib.layers.real_valued_column('x', dimension=1000)]
# #feature_columns = [tf.contrib.layers.real_valued_column("", dimension=4)]

# #cross_entropy = tf.losses.sparse_softmax_cross_entropy(get_train_inputs())
# x_ = tf.placeholder(tf.float64, [1,1000])
# y_ = tf.placeholder(tf.int64, [1,1000])
# w_ = tf.placeholder(tf.int64, [1])

# cross_entropy = tf.losses.sigmoid_cross_entropy(multi_class_labels=y_, logits=x_)


# classifier = DNNEstimator(                                
#                            head = tf.contrib.learn.multi_label_head(n_classes=9,weight_column_name='class_weights',loss_fn=cross_entropy ) , feature_columns=feature_columns,
#                            hidden_units=[2000, 1000, 100],
#                            dropout=0.7,
#                            #model_dir = './model-parallel/seccion_dnn',
#                            model_dir = './tmp2',
#                            config = tf.contrib.learn.RunConfig(save_checkpoints_steps = 500,
#                            save_checkpoints_secs = None)
#                           )

In [15]:
help(tf.losses.sigmoid_cross_entropy)

Help on function sigmoid_cross_entropy in module tensorflow.python.ops.losses.losses_impl:

sigmoid_cross_entropy(multi_class_labels, logits, weights=1.0, label_smoothing=0, scope=None, loss_collection='losses', reduction='weighted_sum_by_nonzero_weights')
    Creates a cross-entropy loss using tf.nn.sigmoid_cross_entropy_with_logits.
    
    `weights` acts as a coefficient for the loss. If a scalar is provided,
    then the loss is simply scaled by the given value. If `weights` is a
    tensor of shape `[batch_size]`, then the loss weights apply to each
    corresponding sample.
    
    If `label_smoothing` is nonzero, smooth the labels towards 1/2:
    
        new_multiclass_labels = multiclass_labels * (1 - label_smoothing)
                                + 0.5 * label_smoothing
    
    Args:
      multi_class_labels: `[batch_size, num_classes]` target integer labels in
        `(0, 1)`.
      logits: Float `[batch_size, num_classes]` logits outputs of the network.
      weights

In [16]:
get_train_inputs()

Paso por aca.........................
[[0.16648076]
 [0.16347616]
 [0.16347616]
 ...
 [0.16648076]
 [0.16347616]
 [0.16648076]]


<function tensorflow.python.estimator.inputs.numpy_io.numpy_input_fn.<locals>.input_fn>

In [17]:
# X_ = data.X_train
# Y_ = data.Y_train
# X_ = X_.todense()
# x = X_
# Y_ = Y_['Seccion'].reset_index(drop=True)
# #print(Y_)
# from sklearn.preprocessing import LabelBinarizer    
# #LabelBinarizer().fit(range(9)).transform(print(Y_))
# #     y = y.apply(remove_accents)
# #     y = y.values

# y = Y_.values

# labels = list( set(y) )

# def to_index(strInput):
#     tmp0 = [0]*len(labels) 
#     tmp0[labels.index(strInput)] = 1
#     return tmp0
#     #print(tmp0)
    
# y_new = list( map(lambda x: to_index(x),y) )

# #print(y_new)
# #print(labels)
#print(labels.find(y[1]))

In [18]:
# def to_index(strInput):
#     tmp0 = [0]*len(labels) 
#     tmp0[labels.index(strInput)] = 1
#     print(tmp0)

# to_index('Cultura')

# Training



In [141]:
validation_monitor = tf.contrib.learn.monitors.ValidationMonitor(
    input_fn= get_test_inputs(),
    every_n_steps=500,
    #early_stopping_metric="accuracy",
    early_stopping_metric="loss",
    early_stopping_metric_minimize=True,
    early_stopping_rounds=5000
    )

#classifier.fit(input_fn=get_train_inputs(), monitors=[validation_monitor], steps=epochs, max_steps=None)
classifier.fit(input_fn=get_train_inputs(), monitors=[validation_monitor], max_steps=None)

weights shape: (47786, 1)
Paso por aca.........................
weights shape: (191145, 1)
[[0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 ...
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 1 0]]


TypeError: 'tuple' object is not callable

# Evaluation performance


In [131]:
def get_test_inputs():
    X_ = data.X_test_dense
    Y_ = data.Y_test
#     X_ = X_.todense()
    x = X_
    Y_ = Y_['Seccion'].reset_index(drop=True)
    y = Y_.values
    
    # computing lengths of each class
    sizes = {label: Y_[Y_ == label].shape[0] for label in labels}
#     print(sizes)

    # creating weights for each sample
    scale_factor = 10e3
    weights = np.asarray([1.0/sizes[label] for label in y])
    weights = scale_factor*weights
    weights = weights[:,np.newaxis]
    print('weights shape: {}'.format(weights.shape))

    labels2 = list( set(y) )
    def to_index(strInput):
        tmp0 = [0]*len(labels2) 
        tmp0[labels2.index(strInput)] = 1
        return tmp0

    y = list( map(lambda x: to_index(x),y) )    
    #dataset = tf.estimator.inputs.numpy_input_fn({'x': x, 'class_weights': weights}, np.array(y), shuffle=False)    
    
    #dataset = tf.estimator.inputs.numpy_input_fn({'x': x, 'class_weights': weights}, y[:,np.newaxis], shuffle=False)
    x = tf.constant(x)
    class_weights =tf.constant(weights) 
    y = tf.constant(np.array(y))
    return {'x':x,'class_weights': class_weights}, y


def get_train_inputs():
    print("Paso por aca.........................")
    X_ = data.X_train
    Y_ = data.Y_train
    X_ = X_.todense()
    x = X_
    Y_ = Y_['Seccion'].reset_index(drop=True)
    #print(Y_)
    from sklearn.preprocessing import LabelBinarizer    
    #LabelBinarizer().fit(range(9)).transform(print(Y_))
#     y = y.apply(remove_accents)
#     y = y.values

    y = Y_.values
    #print(type(y))
    
    # computing lengths of each class
    sizes = {label: Y_[Y_ == label].shape[0] for label in labels}
#     print(sizes)

    # creating weights for each sample
    scale_factor = 10e3
    weights = np.asarray([1.0/sizes[label] for label in y])
    weights = scale_factor*weights
    weights = weights[:,np.newaxis]
    print('weights shape: {}'.format(weights.shape))
    
    
    labels2 = list( set(y) )
    def to_index(strInput):
        tmp0 = [0]*len(labels2) 
        tmp0[labels2.index(strInput)] = 1
        return tmp0

    y = list( map(lambda x: to_index(x),y) )    

    print(np.array(y))
    #print(y[:,np.newaxis])
    
    #dataset = tf.estimator.inputs.numpy_input_fn({'x': x, 'class_weights': weights}, np.array(y), shuffle=True, batch_size=500, num_epochs=epochs)
    
    #dataset = tf.estimator.inputs.numpy_input_fn({'x': x, 'class_weights': weights}, y[:,np.newaxis], shuffle=True, batch_size=500, num_epochs=epochs)
    #return dataset

    x = tf.constant(x)
    class_weights =tf.constant(weights) 
    y = tf.constant(np.array(y))
    return {'x':x,'class_weights': class_weights}, y


In [21]:
# def input_fn_evaluate():
#     dataset = {'x': tf.constant(data.X_test_dense)}
    
#     return dataset

# pred_test = classifier.evaluate(input_fn=get_test_inputs)


score = classifier.evaluate(input_fn=get_test_inputs,
                                    steps=1)


# # print(pred_test)

# y_test_hat = np.asarray([x.decode('UTF-8') for x in list(pred_test)])
# y_test_hat = y_test_hat.astype(str)
# y_test = data.Y_test['Seccion'].values
# y_test = y_test.astype(str)
# print('Test shape: {}\nReal shape: {}'.format(y_test_hat.shape, y_test.shape))

# from sklearn.metrics import accuracy_score

# acc = accuracy_score(y_true=y_test, y_pred=y_test_hat)
# print('Accuracy in test: {}'.format(acc))

print(score)

weights shape: (47786, 1)
INFO:tensorflow:Starting evaluation at 2018-08-10-19:02:54
INFO:tensorflow:Restoring parameters from ./tmp2/model.ckpt-113514
INFO:tensorflow:Evaluation [1/1]
INFO:tensorflow:Finished evaluation at 2018-08-10-19:03:55
INFO:tensorflow:Saving dict for global step 113514: accuracy = 0.8517599, auc = 0.60530096, auc_precision_recall = 0.1733294, global_step = 113514, loss = 0.813079
{'loss': 0.813079, 'accuracy': 0.8517599, 'auc': 0.60530096, 'auc_precision_recall': 0.1733294, 'global_step': 113514}


In [22]:
pred_test = classifier.predict(input_fn=get_test_inputs)


weights shape: (47786, 1)
INFO:tensorflow:Restoring parameters from ./tmp2/model.ckpt-113514


In [23]:
#out = classifier.evaluate(input_fn=get_test_inputs,steps=1)

In [24]:
#print(out.items)

In [78]:

predictions = list(classifier.predict(input_fn = get_test_inputs))
print(predictions[1:10])

weights shape: (47786, 1)
INFO:tensorflow:Restoring parameters from ./tmp2/model.ckpt-113514
[{'logits': array([-5.4730062 , -2.201419  , -2.8765526 , -3.602121  , -3.4763107 ,
       -2.3182328 , -0.71752167, -0.34028864, -2.3252192 ], dtype=float32), 'probabilities': array([0.00418104, 0.09962312, 0.0533249 , 0.02654213, 0.02999383,
       0.08962414, 0.32793894, 0.4157394 , 0.08905575], dtype=float32), 'classes': array([0, 0, 0, 0, 0, 0, 0, 0, 0])}, {'logits': array([-6.2931013 , -3.193989  , -3.5452323 , -4.1725717 , -5.077106  ,
       -3.0382257 , -1.4346077 ,  0.36818933, -2.1173959 ], dtype=float32), 'probabilities': array([0.0018456 , 0.03939255, 0.02805227, 0.01517863, 0.00619926,
       0.04572853, 0.19238175, 0.5910214 , 0.10741749], dtype=float32), 'classes': array([0, 0, 0, 0, 0, 0, 0, 1, 0])}, {'logits': array([-7.2853928 , -4.410593  , -6.4298477 , -3.8925962 , -4.6113157 ,
       -2.8711119 , -2.0688114 , -0.36548015, -0.6972369 ], dtype=float32), 'probabilities': arra

In [71]:
# print(predictions[1:10])

In [52]:
predictions[1:3]["probabilities"]

TypeError: list indices must be integers or slices, not str

In [ ]:
predictions = list(classifier.predict(input_fn = get_test_inputs))


In [136]:
#predictions[1:10]

out = list(map(lambda x:x["classes"] ,predictions))
sums = np.sum(out,axis = 1)
total = sum(sums>=1)
print(total)
print(len(sums))


42175
47786


In [140]:
from sklearn import cross_validation
import sklearn
sklearn.metrics.confusion_matrix(out, out)

ValueError: multiclass-multioutput is not supported

In [74]:
def input_fn_evaluate():
    dataset = {'x': tf.constant(data.X_test_dense)}
    
    return dataset

pred_test = classifier.predict(input_fn=input_fn=get_test_inputs)

# print(pred_test)

y_test_hat = np.asarray([x.decode('UTF-8') for x in list(pred_test)])
y_test_hat = y_test_hat.astype(str)
y_test = data.Y_test['Seccion'].values
y_test = y_test.astype(str)
print('Test shape: {}\nReal shape: {}'.format(y_test_hat.shape, y_test.shape))

from sklearn.metrics import accuracy_score

acc = accuracy_score(y_true=y_test, y_pred=y_test_hat)
print('Accuracy in test: {}'.format(acc))



INFO:tensorflow:Restoring parameters from ./tmp2/model.ckpt-113514


AttributeError: 'dict' object has no attribute 'decode'

In [ ]:

pred_test = list(classifier.predict(input_fn = get_test_inputs))
print(predictions[1:10])

In [145]:
out = list(map(lambda x:x["classes"] ,predictions))


[array([0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 1, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 1, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 1, 0]),
 array([0, 1, 1, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 1, 0]),
 array([0, 0, 1, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0])]

In [148]:
out = list(map(lambda x:x["classes"] ,predictions))
out = np.array(out)

import numpy as np
import itertools
import matplotlib.pyplot as plt

def confusion_matrix(yt, yp, classes):
    instcount = yt.shape[0]
    n_classes = classes.shape[0]
    mtx = np.zeros((n_classes, 4))
    for i in range(instcount):
        for c in range(n_classes):
            mtx[c,0] += 1 if yt[i,c]==1 and yp[i,c]==1 else 0
            mtx[c,1] += 1 if yt[i,c]==1 and yp[i,c]==0 else 0
            mtx[c,2] += 1 if yt[i,c]==0 and yp[i,c]==0 else 0
            mtx[c,3] += 1 if yt[i,c]==0 and yp[i,c]==1 else 0
    mtx = [[m0/(m0+m1), m1/(m0+m1), m2/(m2+m3), m3/(m2+m3)] for m0,m1,m2,m3 in mtx]
    plt.figure(num=None, figsize=(5, 15), dpi=100, facecolor='w', edgecolor='k')
    plt.imshow(mtx, interpolation='nearest',cmap='Blues')
    plt.title("title")
    tick_marks = np.arange(n_classes)
    plt.xticks(np.arange(4), ['1 - 1','1 - 0','0 - 0','0 - 1'])
    plt.yticks(tick_marks, classes)
    for i, j in itertools.product(range(n_classes), range(4)):
        plt.text(j, i, round(mtx[i][j],2), horizontalalignment="center")

    #plt.tight_layout()
    plt.ylabel('labels')
    plt.xlabel('Predicted')
    plt.show()

confusion_matrix(out, out,[1 1 1 1 1 1 1 1 1])
    
# y1 = np.genfromtxt('y_true.csv', delimiter=",")
# y2 = np.genfromtxt('y_pred.csv', delimiter=",")
# labels = np.genfromtxt('labels.csv', delimiter=",", dtype="|S")
# confusion_matrix(y1, y2, labels)

SyntaxError: invalid syntax (<ipython-input-148-18b557612b5d>, line 33)

In [82]:
y_test = data.Y_test['Seccion'].values
y_test = y_test.astype(str)
print(y_test)

['Pais' 'Pais' 'Pais' ... 'Deportes' 'Pais' 'Deportes']


In [83]:
x,y =get_train_inputs()

Paso por aca.........................
weights shape: (191145, 1)
[[0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 ...
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 1 0]]


In [94]:
#with sess.as_default()


TypeError: 'int' object is not callable

In [ ]:
predict_proba = classifier.predict_proba(input_fn=input_fn_evaluate,as_iterable=False)
np.sum(predict_proba, axis=1)

In [ ]:

ax = skplt.plot_confusion_matrix(y_test, y_test_hat,normalize=True)
# plt.xticks(rotation=45)

for tick in ax.xaxis.get_minor_ticks():
    tick.tick1line.set_markersize(0)
    tick.tick2line.set_markersize(0)
    tick.label1.set_horizontalalignment('center')

plt2.xticks(rotation=70)